In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

In [2]:
import pandas as pd

In [8]:
# Read the Redfin data into a DataFrame
file_path_redfin1 = "../data/redfin_data1.csv"
file_path_redfin2 = "../data/redfin_data2.csv"
df_redfin1 = pd.read_csv(file_path_redfin1)
df_redfin2 = pd.read_csv(file_path_redfin2)
df_redfin = pd.concat([df_redfin1, df_redfin2])
df_redfin["period_begin"] = pd.to_datetime(df_redfin["period_begin"])

#make it easier to merge with av_data
df_redfin["date"] = df_redfin["period_begin"] + pd.Timedelta(days = -3)
df_redfin = df_redfin.drop(columns=["period_begin", "period_end"])
df_redfin

,Unnamed: 0,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,...,months_of_supply,months_of_supply_yoy,median_pending_sqft,median_pending_sqft_yoy,average_sale_to_list_ratio,average_sale_to_list_ratio_yoy,median_sale_ppsf,median_sale_ppsf_yoy,State,date
0,2,Crook County,5.0,-0.375000,8.0,0.333333,1.0,-0.500000,9.0,3.500000,...,12.000000,-60.000000,1618.0,-0.071449,0.937917,0.021480,150.000000,0.218853,OR,2017-01-27
1,10,Chambers County,15.0,0.666667,14.0,0.076923,3.0,-0.250000,11.0,0.833333,...,13.363636,-10.469697,2227.0,0.183316,0.973733,-0.015372,94.778849,-0.035945,TX,2017-01-27
2,32,St. Joseph County,12.0,-0.555556,17.0,0.000000,1.0,-0.500000,13.0,-0.133333,...,20.153846,-3.512821,1500.0,-0.417363,0.933705,0.005458,61.458333,0.156474,MI,2017-02-03
3,42,Crook County,6.0,-0.400000,12.0,-0.142857,1.0,-0.500000,1.0,-0.888889,...,107.000000,90.666667,1984.0,0.634267,1.000000,-0.009948,143.509615,0.506180,OR,2017-02-03
4,69,Coos County,16.0,-0.360000,23.0,-0.080000,5.0,-0.166667,11.0,-0.450000,...,28.818182,13.268182,1760.0,0.085415,0.941134,-0.017685,116.892373,-0.041654,OR,2017-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166274,4868262,Troup County,15.0,0.363636,20.0,1.857143,1.0,0.000000,18.0,0.200000,...,14.222222,-3.111111,1648.0,-0.208833,0.988753,-0.010559,146.915071,0.086005,GA,2024-11-15
166275,4868265,Nez Perce County,7.0,0.400000,6.0,0.200000,2.0,-0.333333,8.0,1.000000,...,12.375000,-10.875000,2860.0,0.914324,0.999153,0.041135,189.274448,-0.264456,ID,2024-11-15
166276,4868271,Coles County,17.0,1.833333,11.0,0.222222,4.0,3.000000,9.0,2.000000,...,10.555556,-19.777778,1519.5,-0.038291,0.941275,-0.056837,104.166667,0.351781,IL,2024-11-15
166277,4868277,Marion County,6.0,-0.142857,4.0,-0.500000,4.0,1.000000,12.0,1.400000,...,8.166667,-6.433333,1098.5,-0.147127,0.966881,-0.046709,214.650323,-0.145966,IA,2024-11-15


In [9]:
#Inspect Data
df_redfin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 332558 entries, 0 to 166278
Data columns (total 36 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   Unnamed: 0                                    332558 non-null  int64         
 1   region_name                                   332558 non-null  object        
 2   adjusted_average_new_listings                 332558 non-null  float64       
 3   adjusted_average_new_listings_yoy             332558 non-null  float64       
 4   average_pending_sales_listing_updates         332558 non-null  float64       
 5   average_pending_sales_listing_updates_yoy     332558 non-null  float64       
 6   off_market_in_two_weeks                       332558 non-null  float64       
 7   off_market_in_two_weeks_yoy                   332558 non-null  float64       
 8   adjusted_average_homes_sold                   332558 non-nu

In [5]:
# Read the Zillow data into a DataFrame
file_path_zillow_zori = "../data/County_zori_uc_sfrcondomfr_sm_month.csv"
file_path_zillow_zhvi = "../data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
df_zillow_zori = pd.read_csv(file_path_zillow_zori)
df_zillow_zhvi = pd.read_csv(file_path_zillow_zhvi)

In [6]:
#Extract required features
df_zillow_zori.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"], inplace=True)
df_zillow_zhvi.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"], inplace=True)

#restructure so that the dates become rows
df_zillow_zori = pd.melt(df_zillow_zori, id_vars=["RegionName", "State", "Metro"],
                        var_name="Date",
                        value_name="ZORI")
df_zillow_zhvi = pd.melt(df_zillow_zhvi, id_vars=["RegionName", "State", "Metro"],
                        var_name="Date",
                        value_name="ZHVI")

#apply time horizon
df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)
df_zillow_zhvi = df_zillow_zhvi[df_zillow_zhvi["Date"] > "2015-01-31"]

#merge
df_zillow = pd.merge(df_zillow_zhvi, df_zillow_zori[["RegionName", "State", "Metro", "Date", "ZORI"]],
                     on = ["RegionName", "State", "Metro", "Date"],
                     how = "left")
df_zillow.dropna(inplace=True)

#rename column to match df_redfin
df_zillow.rename(columns={"RegionName": "region_name"}, inplace=True)
df_zillow.rename(columns={"Date": "date"}, inplace=True)

#resample to weekly
df_zillow["date"] += pd.to_timedelta(df_zillow.groupby("date").cumcount(), unit = "s")
df_zillow.set_index("date", inplace=True)
df_zillow = df_zillow.groupby("region_name").resample("W-FRI").ffill()
df_zillow = df_zillow.drop(columns=["region_name"]).reset_index()
df_zillow

<ipython-input-6-a391ab3e0e27>:14: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
<ipython-input-6-a391ab3e0e27>:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)


,region_name,date,State,Metro,ZHVI,ZORI
0,Ada County,2015-03-06,ID,"Boise City, ID",204657.323819,948.248180
1,Ada County,2015-03-13,ID,"Boise City, ID",204657.323819,948.248180
2,Ada County,2015-03-20,ID,"Boise City, ID",204657.323819,948.248180
3,Ada County,2015-03-27,ID,"Boise City, ID",204657.323819,948.248180
4,Ada County,2015-04-03,ID,"Boise City, ID",205924.714086,956.009824
...,...,...,...,...,...,...
250238,Yuma County,2024-10-04,AZ,"Yuma, AZ",277673.650150,1473.600844
250239,Yuma County,2024-10-11,AZ,"Yuma, AZ",277673.650150,1473.600844
250240,Yuma County,2024-10-18,AZ,"Yuma, AZ",277673.650150,1473.600844
250241,Yuma County,2024-10-25,AZ,"Yuma, AZ",277673.650150,1473.600844


In [7]:
#Inspect Data
df_zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250243 entries, 0 to 250242
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   region_name  250243 non-null  object        
 1   date         250243 non-null  datetime64[ns]
 2   State        250198 non-null  object        
 3   Metro        250198 non-null  object        
 4   ZHVI         250198 non-null  float64       
 5   ZORI         250198 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 11.5+ MB


In [4]:
#Read alpha vantage data
file_path_av = "../data/av_data.csv"
df_av = pd.read_csv(file_path_av)
df_av["date"] = pd.to_datetime(df_av["date"])
df_av

,date,SPY_adj_close,EQR_adj_close,EQR_adj_close_1week,EQR_adj_close_1month,EQR_adj_close_3month,ESS_adj_close,ESS_adj_close_1week,ESS_adj_close_1month,ESS_adj_close_3month,...,AVB_adj_close_1month,AVB_adj_close_3month,INVH_adj_close,INVH_adj_close_1week,INVH_adj_close_1month,INVH_adj_close_3month,unemployment,fed_funds,3monhth_treasury_yield,cpi
0,2017-02-10,203.3163,47.0301,46.7643,47.0528,50.1303,177.5320,177.3143,174.3215,196.0941,...,137.0456,149.7439,17.5995,17.7084,17.9177,17.8088,0.046,0.000104,0.0066,2.43603
1,2017-02-17,206.4603,46.7643,47.6756,48.1692,49.6786,177.3143,180.6181,181.2400,197.8955,...,143.2823,149.1924,17.7084,18.0851,18.0851,18.2942,0.046,0.000102,0.0066,2.43603
2,2017-02-24,207.9094,47.6756,47.3262,48.1783,50.1762,180.6181,179.2188,183.9607,199.3679,...,143.8682,148.5786,18.0851,18.2610,18.0684,18.2858,0.046,0.000100,0.0066,2.43603
3,2017-03-03,209.3848,47.3262,47.0528,47.6272,49.8853,179.2188,174.3215,181.3383,200.7385,...,142.6355,148.6330,18.2610,17.9177,18.2777,17.7989,0.046,0.000117,0.0065,2.43603
4,2017-03-10,208.7437,47.0528,48.1692,48.2549,51.0718,174.3215,181.2400,182.8185,205.6963,...,143.5444,151.9891,17.9177,18.0851,18.0935,18.4117,0.044,0.000142,0.0066,2.43801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,2024-08-16,552.6125,70.7754,72.9757,77.3861,73.5300,281.6740,293.6546,311.4669,304.6600,...,232.9916,233.7500,35.0001,35.8632,36.7263,33.7000,0.042,0.000999,0.0533,3.14796
381,2024-08-23,560.4086,72.9757,74.2146,75.8994,73.5900,293.6546,299.3078,303.6914,301.7600,...,228.3367,229.4100,35.8632,36.5477,35.7540,33.7900,0.042,0.000990,0.0533,3.14796
382,2024-08-30,561.9538,74.2146,73.6794,73.9700,75.7900,299.3078,295.7572,292.1769,305.5300,...,223.5429,231.9400,36.5477,35.6648,35.0000,33.8800,0.042,0.000979,0.0533,3.14796
383,2024-09-06,538.7052,73.6794,77.3861,73.5300,76.6600,295.7572,311.4669,289.8800,310.4600,...,221.0200,235.3500,35.6648,36.7263,34.5400,34.2500,0.041,0.000968,0.0533,3.15301


In [114]:
#Inspect Data
df_av.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          400 non-null    datetime64[ns]
 1   spy_price     400 non-null    float64       
 2   schh_price    400 non-null    float64       
 3   unemployment  400 non-null    float64       
 4   fed_funds     400 non-null    float64       
 5   cpi           400 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 18.9 KB


In [10]:
#merge three data source
df = pd.merge(df_redfin, df_zillow[["region_name", "State", "date" , "ZORI", "ZHVI"]],
                     on = ["region_name", "State", "date"],
                     how = "left")

df = pd.merge(df, df_av,
                     on = [ "date"],
                     how = "left")
df.dropna(inplace=True)
df

,Unnamed: 0,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,...,AVB_adj_close_1month,AVB_adj_close_3month,INVH_adj_close,INVH_adj_close_1week,INVH_adj_close_1month,INVH_adj_close_3month,unemployment,fed_funds,3monhth_treasury_yield,cpi
16,274,Stanislaus County,163.0,0.108844,152.0,-0.061728,52.0,-0.187500,112.0,-0.157895,...,143.5444,151.9891,17.9177,18.0851,18.0935,18.4117,0.044,0.000142,0.0066,2.43801
18,286,Stearns County,73.0,0.123077,42.0,0.000000,11.0,0.100000,31.0,0.033333,...,143.5444,151.9891,17.9177,18.0851,18.0935,18.4117,0.044,0.000142,0.0066,2.43801
38,507,Denver County,335.0,-0.079670,313.0,-0.163102,162.0,0.045161,341.0,0.249084,...,147.9058,142.9288,17.9080,17.7653,19.2879,19.3131,0.043,0.000216,0.0116,2.44786
45,611,Stanislaus County,120.0,-0.097744,165.0,0.071429,44.0,-0.185185,76.0,-0.296296,...,143.2823,149.1924,17.7084,18.0851,18.0851,18.2942,0.046,0.000102,0.0066,2.43603
47,623,Stearns County,50.0,-0.107143,46.0,0.210526,6.0,0.500000,16.0,-0.238095,...,143.2823,149.1924,17.7084,18.0851,18.0851,18.2942,0.046,0.000102,0.0066,2.43603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332534,4868129,Cuyahoga County,397.0,-0.076744,405.0,-0.093960,150.0,-0.202128,292.0,-0.165714,...,200.3976,221.9251,34.9415,34.8627,35.0894,35.6648,0.041,0.001032,0.0533,3.14175
332536,4868136,Butler County,14.0,-0.300000,15.0,-0.062500,6.0,0.200000,20.0,0.250000,...,221.9251,216.2800,35.4068,35.0001,35.6648,31.4000,0.042,0.001001,0.0533,3.14796
332539,4868155,Schuylkill County,40.0,0.333333,34.0,0.096774,12.0,-0.076923,30.0,0.200000,...,200.3976,221.9251,34.9415,34.8627,35.0894,35.6648,0.041,0.001032,0.0533,3.14175
332540,4868161,Aransas County,14.0,0.400000,14.0,0.555556,0.0,-1.000000,16.0,1.000000,...,200.3976,221.9251,34.9415,34.8627,35.0894,35.6648,0.041,0.001032,0.0533,3.14175


In [11]:
df.describe()

,Unnamed: 0,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_new_listing_price,...,AVB_adj_close_1month,AVB_adj_close_3month,INVH_adj_close,INVH_adj_close_1week,INVH_adj_close_1month,INVH_adj_close_3month,unemployment,fed_funds,3monhth_treasury_yield,cpi
count,1.254710e+05,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,1.254710e+05,...,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000,125471.000000
mean,2.792281e+06,144.961401,0.021727,147.424656,0.005612,39.795674,0.235339,124.017924,0.005686,3.739410e+05,...,172.078953,174.194623,28.711928,28.752578,28.864495,29.101577,0.044226,0.000515,0.026065,2.803210
min,2.740000e+02,1.000000,-0.956349,1.000000,-0.995088,0.000000,-1.000000,1.000000,-0.941176,2.657100e+04,...,111.523100,111.523100,15.199100,15.199100,15.199100,15.199100,0.034000,0.000002,0.000400,2.436030
25%,1.662256e+06,38.000000,-0.172478,40.000000,-0.183673,7.000000,-0.333333,33.000000,-0.200000,2.400000e+05,...,150.652700,151.752000,23.570400,23.675000,23.893000,24.399800,0.036000,0.000149,0.007400,2.563940
50%,2.974993e+06,79.000000,-0.018182,81.000000,-0.037639,18.000000,0.000000,69.000000,-0.038462,3.224500e+05,...,171.705900,172.035200,30.442900,30.442900,30.562600,30.711900,0.038000,0.000420,0.021800,2.788020
75%,3.999820e+06,166.000000,0.147059,171.000000,0.122372,45.000000,0.400000,145.000000,0.138462,4.374500e+05,...,189.031600,192.296700,33.901300,33.937500,33.991500,33.991500,0.043000,0.001006,0.050800,3.056910
max,4.868184e+06,2608.000000,22.818182,2649.000000,154.000000,1165.000000,57.000000,2679.000000,16.300000,1.075000e+07,...,232.991600,235.350000,41.540000,40.266500,41.540000,41.540000,0.148000,0.001052,0.053300,3.153010
std,1.380067e+06,215.912657,0.365836,217.382199,0.669920,65.270243,1.359944,180.293705,0.360771,2.243765e+05,...,26.742999,28.368430,6.451726,6.437393,6.436596,6.307238,0.017398,0.000401,0.020724,0.250246
